# The chess interface and game loop

### Dependencies

In [ ]:
%pip install python-chess # TODO Remove

import chess, chess.svg, chess.polyglot
import time
import random
from IPython.display import clear_output, SVG, display, HTML


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### chess.Board.push_legal
Pushes a move to a board's move stack only if it is a valid, legal move. If the move is illegal or invalid, a ValueError is raised and the board state remains unaffected.

##### Arguments
    move : chess.Move
        The move that should be pushed to the move stack.

##### Returns
    N/A

##### Side effects
    - Pushes a move to the board's move stack, assuming the move was valid and legal.

In [ ]:
def push_legal(self, move: chess.Move):
    if move not in self.legal_moves:
        raise ValueError('Illegal move')
    self.push(move)

chess.Board.push_legal = push_legal

---
<br>

## Game class
Lets the user play a game of chess against the AI using a virtual chess board UI. The game may use any algorithm for the AI to make its move.
<br><br>

### Class variables
    board : chess.Board
        The chess board instance that's used for the game.
    make_move : function(chess.Board)
        The function that the AI uses to make a new move.
<br>

### \_\_init\_\_
    The constructor for the Game class. Gets called whenever a new Game instance is created.
<br>

##### Arguments
    make_move_algorithm : function(chess.Board)
        The function that the AI should use to make a new move.

##### Returns
    N/A

##### Side effects
    N/A
<br><br>

### play
    Plays the game, which includes three features:
    - Displays the chess board visually.
    - Lets the AI make a move based on the board state.
    - Lets the human player input their move and pushes it to the board (if it's valid and legal).

##### Arguments
    N/A

##### Returns
    N/A

##### Side effects
    - The state of the board in the Game class is changed.
<br><br>

### display
    Displays the chess board graphically.

##### Arguments
    html : str
        HTML content that should be displayed next to the board.
    header : str
        Text to display at the top of the HTML content.
    board_size : int
        The size of the board (both width and height), in pixels.

##### Returns
    N/A

##### Side effects
    - The current visual output is overwritten.

In [ ]:
class Gam2e:
    board = None
    make_move = None

    def __init__(self, make_move_algorithm):
        self.board = chess.Board()
        self.make_move = make_move_algorithm

    def play(self):
        while not self.board.is_game_over():
            self.make_move(self.board)
            self.display("html stuff", "Header")

            input_prompt = 'Please input your move: '
            while True:
                try:
                    move = chess.Move.from_uci(input(input_prompt))
                    self.board.push_legal(move)
                    break
                except ValueError:
                    input_prompt = 'Illegal move, please try again: '
            
            self.display("html stuff", "Header")
            

    def display(self, html: str, header: str, board_size: int = 500):
        clear_output(wait=True)
        board_visual = None
        try:
            last_move = self.board.peek()
            board_visual = chess.svg.board(self.board, size=board_size, lastmove=last_move) 
        except IndexError:
            board_visual = chess.svg.board(self.board, size=board_size)
        table = '<table><th>Board</th><th>{}</th><tr><td><div style=\"vertical-align: top; text-align: left\">{}</div></td><td>{}</td></div></tr></table>'
        display(HTML(table.format(header, board_visual, html)))



In [ ]:
class Game:
    board = None
    make_move = None
    is_ai = None
    enemy_ai = None
    move_times = None
    opening_book = None
    is_first_move = True

    def __init__(self, make_move_algorithm, is_ai, enemy_ai=None, opening_book=None):
        self.board = chess.Board()
        self.move_times = { 'white': [], 'black': [] }
        self.is_first_move = True
        self.make_move = make_move_algorithm
        self.enemy_ai = enemy_ai 
        self.is_ai = is_ai
        self.opening_book = opening_book

    def play(self):
        while not self.board.is_game_over():
            # Start time [white]
            time_stamp = time.time()
            if not self.make_opening_move():
                self.make_move(self.board)
            # Find time difference
            time_stamp = time.time() - time_stamp
            #Add time to white_move
            self.move_times['white'].append(time_stamp)

            #display board
            self.display("html stuff", "Header")
            if self.is_ai['black'] != True:
                input_prompt = 'Please input your move: '
                while not self.board.is_game_over():
                    try:
                        move = chess.Move.from_uci(input(input_prompt))
                        self.board.push_legal(move)
                        break
                    except ValueError:
                        input_prompt = 'Illegal move, please try again: '
                
                    self.display("html stuff", "Header")
            else:
                while not self.board.is_game_over():
                    try:
                        #Start time [black]
                        time_stamp = time.time()
                        self.enemy_ai(self.board)
                        #Find time difference
                        time_stamp = time.time() - time_stamp
                        #Add time to black_moves
                        self.move_times['black'].append(time_stamp)
                        
                        #display board
                        self.display("html stuff", "Header")
                        break
                    except Exception as e:
                        print("Unkown error, we are sorry!")
                        print(e)
        print(self.board.outcome())


            

    def display(self, html: str, header: str, board_size: int = 500):
        clear_output(wait=True)
        board_visual = None
        try:
            last_move = self.board.peek()
            board_visual = chess.svg.board(self.board, size=board_size, lastmove=last_move, check=self.check_check()) 
        except IndexError:
            board_visual = chess.svg.board(self.board, size=board_size)
        table = '<table><th>Board</th><th>{}</th><tr><td><div style=\"vertical-align: top; text-align: left\">{}</div></td><td>{}</td></div></tr></table>'
        display(HTML(table.format(header, board_visual, html)))

    def check_check(self):
        if self.board.is_check():
            return self.board.checkers().pop()
        else:
            return None

    def make_opening_move(self):
        if not self.is_first_move or self.opening_book is None:
            self.is_first_move = False
            return False

        with chess.polyglot.open_reader(self.opening_book) as reader:
            possible_moves = [ entry.move for entry in reader.find_all(self.board) ]
            self.board.push(random.choice(possible_moves))

        self.is_first_move = False
        return True
            


In [ ]:
class GameTest:
    board = None
    make_move = None
    is_ai = None
    enemy_ai = None # Rename
    move_times = None
    opening_book = None
    is_first_move = True

    def __init__(self, make_move_algorithm, is_ai, enemy_ai=None, opening_book=None):
        self.board = chess.Board()
        self.move_times = { 'white': [], 'black': [] }
        self.is_first_move = True
        self.make_move = make_move_algorithm
        self.enemy_ai = enemy_ai 
        self.is_ai = is_ai
        self.opening_book = opening_book

    def play(self):
        while not self.board.is_game_over():
            # Start time [white]
            time_stamp = time.time()
            if not self.make_opening_move():
                self.make_move(self.board)
            # Find time difference
            time_stamp = time.time() - time_stamp
            #Add time to white_move
            self.move_times['white'].append(time_stamp)

            #display board
            self.display("html stuff", "Header")
            if self.is_ai['black'] != True:
                input_prompt = 'Please input your move: '
                while not self.board.is_game_over():
                    try:
                        move = chess.Move.from_uci(input(input_prompt))
                        self.board.push_legal(move)
                        break
                    except ValueError:
                        input_prompt = 'Illegal move, please try again: '
                
                    self.display("html stuff", "Header")
            else:
                while not self.board.is_game_over():
                    try:
                        #Start time [black]
                        time_stamp = time.time()
                        self.enemy_ai(self.board)
                        #Find time difference
                        time_stamp = time.time() - time_stamp
                        #Add time to black_moves
                        self.move_times['black'].append(time_stamp)
                        
                        #display board
                        self.display("html stuff", "Header")
                        break
                    except Exception as e:
                        print("Unkown error, we are sorry!")
                        print(e)
        print(self.board.outcome())


            

    def display(self, html: str, header: str, board_size: int = 500):
        clear_output(wait=True)
        board_visual = None
        try:
            last_move = self.board.peek()
            board_visual = chess.svg.board(self.board, size=board_size, lastmove=last_move, check=self.check_check()) 
        except IndexError:
            board_visual = chess.svg.board(self.board, size=board_size)
        table = '<table><th>Board</th><th>{}</th><tr><td><div style=\"vertical-align: top; text-align: left\">{}</div></td><td>{}</td></div></tr></table>'
        display(HTML(table.format(header, board_visual, html)))

    def check_check(self):
        if self.board.is_check():
            return self.board.checkers().pop()
        else:
            return None

    def make_opening_move(self):
        if not self.is_first_move or self.opening_book is None:
            self.is_first_move = False
            return False

        with chess.polyglot.open_reader(self.opening_book) as reader:
            print([ entry.move for entry in reader.find_all(self.board) ])
            time.sleep(5)
            self.board.push(random.choice([ entry.move for entry in reader.find_all(self.board) ]))
            # for entry in reader.find_all(self.board):
            #     print(entry, entry.move, entry.weight, entry.learn)

        self.is_first_move = False
        return True
            


In [ ]:
class Game3:
    board = None
    make_move_white = None
    is_ai = { 'white': None, 'black': None }
    make_move_black = None
    player = None

    def __init__(self, make_move_algorithm_white = None, make_move_algorithm_black = None):
        self.board = chess.Board()
        self.make_move_white = make_move_algorithm_white
        self.make_move_black = make_move_algorithm_black 
        self.is_ai, self.player = self.get_player_state()

    def play(self):

        if self.make_move_black is None and self.make_move_white is None:
            print("Exiting game, no AI detected!")
            return None

        while not self.board.is_game_over():
           
            if self.player == 'black':
                print("Player is black")
                while not self.board.is_game_over():
                    self.make_move_white(self.board)
                    self.display("html stuff", "Header")
                    input_prompt = 'Please input your move: '
                    try:
                        move = chess.Move.from_uci(input(input_prompt))
                        self.board.push_legal(move)
                        
                    except ValueError:
                        input_prompt = 'Illegal move, please try again: '
                
                    self.display("html stuff", "Header")

            elif self.player == 'white':
                while not self.board.is_game_over():
                    self.display("html stuff", "Header")
                    input_prompt = 'Please input your move: '
                    move = chess.Move.from_uci(input(input_prompt))
                    self.board.push_legal(move)
                    self.display("html stuff", "Header")
                    self.make_move_black(self.board)
                    self.display("html stuff", "Header")


                       
                
            
            
            
            else:
                while not self.board.is_game_over():
                    try:
                        self.make_move_white(self.board)
                        self.display("html stuff", "Header")
                        self.make_move_black(self.board)
                        self.display("html stuff", "Header")
                        
                    except Exception as e:
                        print("Unkown error, we are sorry!")
                        print(e)
        
        print(self.board.outcome())


            

    def display(self, html: str, header: str, board_size: int = 500):
        clear_output(wait=True)
        board_visual = None
        try:
            last_move = self.board.peek()
            board_visual = chess.svg.board(self.board, size=board_size, lastmove=last_move, check=self.check_check()) 
        except IndexError:
            board_visual = chess.svg.board(self.board, size=board_size)
        table = '<table><th>Board</th><th>{}</th><tr><td><div style=\"vertical-align: top; text-align: left\">{}</div></td><td>{}</td></div></tr></table>'
        display(HTML(table.format(header, board_visual, html)))

    def check_check(self):
        if self.board.is_check():
            return self.board.checkers().pop()
        else:
            return None

    def get_player_state(self):
        while True:
            if self.make_move_black is not None and self.make_move_white is not None:
                return {'white': True, 'black': True},  None
            elif self.make_move_black is not None and self.make_move_white is None:
                return {'white': False, 'black': True}, 'white'
            elif self.make_move_black is None and self.make_move_white is not None:
                return {'white': True, 'black': False}, 'black'
        
            


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>